# CSV Analyzer


# SetUp

1. Install the Langchain packages: Open a terminal window and enter the command below. You can also [install it in a virtualenv](https://googleapis.dev/python/aiplatform/latest/index.html)

In [2]:
#!pip install --upgrade google-cloud-aiplatform
!pip install langchain-google-genai
!pip install langchain
!pip install langchain-experimental
!pip install langchain-google-vertexai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.8/160.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.0/760.0 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.0/299.0 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.6
    U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.1/208.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 2.8.0
    Uninstalling google-cloud-storage-2.8.0:
      Successfully uninstalled google-cloud-storage-2.8.0


## Download File

In [3]:
from google.cloud import storage


# Replace with your actual bucket and file information
bucket_name = 'felipe-sandbox-354619-bucket'
file_name = 'ShipperrNPSfile_Sept-Aug2024(APAC)_SP-1K.csv'
destination_file_name = 'downloaded_data.csv'

# Initialize a GCS client
storage_client = storage.Client()

# Get the bucket and blob (file)
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(file_name)

# Download the file to your local system
blob.download_to_filename(destination_file_name)

Tools

In [29]:
from langchain.agents import Tool
from langchain.tools import tool
import pandas as pd
from io import StringIO
import pandas as pd
from langchain.pydantic_v1 import BaseModel, Field


@tool("pandas_calculator", return_direct=False)
def pandas_calculator(expr:str) -> pd.DataFrame:
    """
    Evaluates an expression on a Pandas DataFrame and returns the result.
    """
    try:
        # evaluate expr on the pandas dataframe
        result = eval(expr)
        # return the result
        return result
    except Exception as e:
        return f"Error: Could not evaluate expression. {e}"

In [14]:
# prompt: create a langchain tool that takes a file as input and loads it into a pandas dataframe and returns the dataframe

import pandas as pd
@tool("load_csv_tool", return_direct=False)
def load_csv_tool(csv_file_path: str) -> pd.DataFrame:
  """
  Loads a CSV file into a Pandas DataFrame.
  """
  try:
    df = pd.read_csv(csv_file_path)
    return df
  except Exception as e:
    return f"Error: Could not load CSV file. {e}"



In [30]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_google_vertexai import ChatVertexAI
from langchain.memory import ConversationBufferWindowMemory


llm = ChatVertexAI(model="gemini-1.5-pro-002")
prompt = hub.pull("hwchase17/react-chat")
chat_history = ConversationBufferWindowMemory(k=10)


#@tool("data_fill_tool", args_schema=DataFillTool, return_direct=False)
#def data_fill_tool(input:str) -> str:
#    '''transforms the data based on instructions.'''

#    return ({"text" : "A, B , C \n 1, 2, 3 \n"})

agent = create_react_agent(llm, [pandas_calculator,load_csv_tool], prompt)
agent_executor = AgentExecutor(agent=agent,
                               tools=[pandas_calculator,load_csv_tool],
                               verbose=True, handle_parsing_errors=True,
                               max_iterations=5)


def append_chat_history(input, response):
    chat_history.save_context({"input": input}, {"output": response})

def invoke(input):
    msg = {
        "input": input,
        "chat_history": chat_history.load_memory_variables({}),
    }
    print(f"Input: {msg}")

    response = agent_executor.invoke(msg)
    print(f"Response: {response}")

    append_chat_history(response["input"], response["output"])
    print(f"History: {chat_history.load_memory_variables({})}")

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:221: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [31]:
# read downloaded csv to pandas df
df = pd.read_csv(destination_file_name)

# convert df to string
df_str = df.to_string(index=False)
invoke("Hi")
invoke(destination_file_name + "\n\n how many rows are in this file?")

Input: {'input': 'Hi', 'chat_history': {'history': ''}}


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: Hello! How can I help you today?
 How can I help you today?


> Finished chain.
Response: {'input': 'Hi', 'chat_history': {'history': ''}, 'output': 'Hello! How can I help you today?\n How can I help you today?'}
History: {'history': 'Human: Hi\nAI: Hello! How can I help you today?\n How can I help you today?'}
Input: {'input': 'downloaded_data.csv\n\n how many rows are in this file?', 'chat_history': {'history': 'Human: Hi\nAI: Hello! How can I help you today?\n How can I help you today?'}}


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: load_csv_tool
Action Input: downloaded_data.csv     Index                                           Comments
0        1  The service is perfect, the delivery is timely...
1        2  At present, among the orders sent by UPS expre...
2        3                           

In [32]:
invoke("can you add the numbers in the Index column?")

Input: {'input': 'can you add the numbers in the Index column?', 'chat_history': {'history': 'Human: Hi\nAI: Hello! How can I help you today?\n How can I help you today?\nHuman: downloaded_data.csv\n\n how many rows are in this file?\nAI: The file has 1000 rows.\n1000 rows.'}}


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: load_csv_tool
Action Input: downloaded_data.csv     Index                                           Comments
0        1  The service is perfect, the delivery is timely...
1        2  At present, among the orders sent by UPS expre...
2        3                                        too rubbish
3        4  The investigation of the transport package was...
4        5  The customs declarer can't handle the return a...
..     ...                                                ...
995    996  The reason why I choose UPS now is because the...
996    997  The collect service is very good. Sometimes th...
997    998                     

In [33]:
invoke("how many rows include the word UPS?")

Input: {'input': 'how many rows include the word UPS?', 'chat_history': {'history': "Human: Hi\nAI: Hello! How can I help you today?\n How can I help you today?\nHuman: downloaded_data.csv\n\n how many rows are in this file?\nAI: The file has 1000 rows.\n1000 rows.\nHuman: can you add the numbers in the Index column?\nAI: The sum of the numbers in the 'Index' column is 500500."}}


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: load_csv_tool
Action Input: downloaded_data.csv     Index                                           Comments
0        1  The service is perfect, the delivery is timely...
1        2  At present, among the orders sent by UPS expre...
2        3                                        too rubbish
3        4  The investigation of the transport package was...
4        5  The customs declarer can't handle the return a...
..     ...                                                ...
995    996  The reason why I choose UPS now is bec

In [37]:
invoke("how many rows are positive and how many are negative?")

Input: {'input': 'how many rows are positive and how many are negative?', 'chat_history': {'history': 'Human: Hi\nAI: Hello! How can I help you today?\n How can I help you today?\nHuman: downloaded_data.csv\n\n how many rows are in this file?\nAI: The file has 1000 rows.\n1000 rows.\nHuman: can you add the numbers in the Index column?\nAI: The sum of the numbers in the \'Index\' column is 500500.\nHuman: how many rows include the word UPS?\nAI: There are 100 rows that include the word "UPS".\n00 rows that include the word "UPS".'}}


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: load_csv_tool
Action Input: downloaded_data.csv     Index                                           Comments
0        1  The service is perfect, the delivery is timely...
1        2  At present, among the orders sent by UPS expre...
2        3                                        too rubbish
3        4  The investigation of the transport package was...
4        5  The cus